In [1]:
# Import necessary libraries
import os
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio
from IPython import display
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, BatchNormalization, Dropout
from keras.losses import SparseCategoricalCrossentropy
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow import keras
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [3]:
# Define the bird dataset CSV file path
bird_csv_path = '.\small_birds_dataset\metadata.csv'  # Replace with the actual path to your Birds dataset CSV file

# Load the bird dataset CSV file
bird_pd_data = pd.read_csv(bird_csv_path)

# Display the first few rows of the bird dataset
bird_pd_data.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,date,filename,license,rating,time,url,duration,split
0,azaspi1,[],['song'],8.7171,-71.33040,Synallaxis azarae,Azara's Spinetail,Joe Klaiber,0000-00-00,XC219896.wav,Creative Commons Attribution-NonCommercial-Sha...,5.0,00:00,https://www.xeno-canto.org/219896,9.682,train
1,azaspi1,[],['song'],-0.4400,-77.86240,Synallaxis azarae,Azara's Spinetail,GABRIEL LEITE,2014-07-11,XC186641.wav,Creative Commons Attribution-NonCommercial-Sha...,5.0,11:00,https://www.xeno-canto.org/186641,12.304,train
2,azaspi1,[],['song'],-17.1528,-65.90840,Synallaxis azarae,Azara's Spinetail,Sjoerd Mayer,1992-08-28,XC1596.wav,Creative Commons Attribution-NonCommercial-Sha...,5.0,?,https://www.xeno-canto.org/1596,12.862,train
3,azaspi1,[],['song'],-4.6476,-79.74426,Synallaxis azarae,Azara's Spinetail,Willem-Pier Vellinga,2000-07-00,XC746.wav,Creative Commons Attribution-NonCommercial-Sha...,4.5,?:?,https://www.xeno-canto.org/746,14.358,test
4,azaspi1,[],['song'],1.6564,-76.18300,Synallaxis azarae,Azara's Spinetail,David Bradley,2015-12-26,XC298616.wav,Creative Commons Attribution-NonCommercial-Sha...,5.0,06:00,https://www.xeno-canto.org/298616,14.780,train


In [4]:
# Load your Birds dataset
birds_csv = 'small_birds_dataset/metadata.csv'
base_data_path = 'small_birds_dataset'

# Read CSV with the provided column names
pd_data = pd.read_csv(birds_csv)

# Define your classes and map class to ID
my_classes = ['White-breasted Wood-Wren', 'House Sparrow', 'Red Crossbill', 'Chestnut-crowned Antpitta', 'Azara\'s Spinetail']
map_class_to_id = {bird_class: idx for idx, bird_class in enumerate(my_classes)}

# Filter the dataframe based on your classes
filtered_pd = pd_data[pd_data['common_name'].isin(my_classes)]

# Map bird species to class ID
class_id = filtered_pd['common_name'].apply(lambda name: map_class_to_id[name])
filtered_pd = filtered_pd.assign(target=class_id)

# Create full paths for audio files
full_path = filtered_pd.apply(lambda row: os.path.join(base_data_path, row['split'], row['primary_label'], row['filename']), axis=1)
filtered_pd = filtered_pd.assign(filename=full_path)

# Display the modified dataframe
filtered_pd.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,date,filename,license,rating,time,url,duration,split,target
0,azaspi1,[],['song'],8.7171,-71.33040,Synallaxis azarae,Azara's Spinetail,Joe Klaiber,0000-00-00,small_birds_dataset\train\azaspi1\XC219896.wav,Creative Commons Attribution-NonCommercial-Sha...,5.0,00:00,https://www.xeno-canto.org/219896,9.682,train,4
1,azaspi1,[],['song'],-0.4400,-77.86240,Synallaxis azarae,Azara's Spinetail,GABRIEL LEITE,2014-07-11,small_birds_dataset\train\azaspi1\XC186641.wav,Creative Commons Attribution-NonCommercial-Sha...,5.0,11:00,https://www.xeno-canto.org/186641,12.304,train,4
2,azaspi1,[],['song'],-17.1528,-65.90840,Synallaxis azarae,Azara's Spinetail,Sjoerd Mayer,1992-08-28,small_birds_dataset\train\azaspi1\XC1596.wav,Creative Commons Attribution-NonCommercial-Sha...,5.0,?,https://www.xeno-canto.org/1596,12.862,train,4
3,azaspi1,[],['song'],-4.6476,-79.74426,Synallaxis azarae,Azara's Spinetail,Willem-Pier Vellinga,2000-07-00,small_birds_dataset\test\azaspi1\XC746.wav,Creative Commons Attribution-NonCommercial-Sha...,4.5,?:?,https://www.xeno-canto.org/746,14.358,test,4
4,azaspi1,[],['song'],1.6564,-76.18300,Synallaxis azarae,Azara's Spinetail,David Bradley,2015-12-26,small_birds_dataset\train\azaspi1\XC298616.wav,Creative Commons Attribution-NonCommercial-Sha...,5.0,06:00,https://www.xeno-canto.org/298616,14.780,train,4


In [5]:
filtered_pd

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,date,filename,license,rating,time,url,duration,split,target
0,azaspi1,[],['song'],8.7171,-71.33040,Synallaxis azarae,Azara's Spinetail,Joe Klaiber,0000-00-00,small_birds_dataset\train\azaspi1\XC219896.wav,Creative Commons Attribution-NonCommercial-Sha...,5.0,00:00,https://www.xeno-canto.org/219896,9.682,train,4
1,azaspi1,[],['song'],-0.4400,-77.86240,Synallaxis azarae,Azara's Spinetail,GABRIEL LEITE,2014-07-11,small_birds_dataset\train\azaspi1\XC186641.wav,Creative Commons Attribution-NonCommercial-Sha...,5.0,11:00,https://www.xeno-canto.org/186641,12.304,train,4
2,azaspi1,[],['song'],-17.1528,-65.90840,Synallaxis azarae,Azara's Spinetail,Sjoerd Mayer,1992-08-28,small_birds_dataset\train\azaspi1\XC1596.wav,Creative Commons Attribution-NonCommercial-Sha...,5.0,?,https://www.xeno-canto.org/1596,12.862,train,4
3,azaspi1,[],['song'],-4.6476,-79.74426,Synallaxis azarae,Azara's Spinetail,Willem-Pier Vellinga,2000-07-00,small_birds_dataset\test\azaspi1\XC746.wav,Creative Commons Attribution-NonCommercial-Sha...,4.5,?:?,https://www.xeno-canto.org/746,14.358,test,4
4,azaspi1,[],['song'],1.6564,-76.18300,Synallaxis azarae,Azara's Spinetail,David Bradley,2015-12-26,small_birds_dataset\train\azaspi1\XC298616.wav,Creative Commons Attribution-NonCommercial-Sha...,5.0,06:00,https://www.xeno-canto.org/298616,14.780,train,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,wbwwre1,[],['song'],-0.9415,-77.89890,Henicorhina leucosticta,White-breasted Wood-Wren,Leonardo Ordóñez-Delgado,2011-12-10,small_birds_dataset\train\wbwwre1\XC116238.wav,Creative Commons Attribution-NonCommercial-Sha...,5.0,17:00,https://www.xeno-canto.org/116238,99.102,train,0
290,wbwwre1,[],['song'],0.8670,-78.73300,Henicorhina leucosticta,White-breasted Wood-Wren,Niels Krabbe,1999-10-21,small_birds_dataset\train\wbwwre1\XC242770.wav,Creative Commons Attribution-ShareAlike 4.0,5.0,10:35:00 AM,https://www.xeno-canto.org/242770,113.526,train,0
291,wbwwre1,[],['song'],8.5054,-77.97590,Henicorhina leucosticta,White-breasted Wood-Wren,Kent Livezey,2017-05-25,small_birds_dataset\train\wbwwre1\XC372481.wav,Creative Commons Attribution-NonCommercial-Sha...,5.0,09:00,https://www.xeno-canto.org/372481,120.780,train,0
292,wbwwre1,[],['song'],-3.7490,-78.87060,Henicorhina leucosticta,White-breasted Wood-Wren,Leonardo Ordóñez-Delgado,2014-06-24,small_birds_dataset\train\wbwwre1\XC190204.wav,Creative Commons Attribution-NonCommercial-Sha...,5.0,17:00,https://www.xeno-canto.org/190204,127.122,train,0


In [6]:
# Create TensorFlow Dataset for training
train_filenames = filtered_pd[filtered_pd['split'] == 'train']['filename']
train_targets = filtered_pd[filtered_pd['split'] == 'train']['target']

train_ds = tf.data.Dataset.from_tensor_slices((train_filenames, train_targets))

In [7]:
train_ds

<TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [8]:
# Create TensorFlow Dataset for testing
test_filenames = filtered_pd[filtered_pd['split'] == 'test']['filename']
test_targets = filtered_pd[filtered_pd['split'] == 'test']['target']

test_ds = tf.data.Dataset.from_tensor_slices((test_filenames, test_targets))

In [27]:
test_ds

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 1024), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [33]:
# Define the load_wav_16k_mono function
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
        file_contents,
        desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)

    # Resample using tf.audio.resample
    target_sample_rate = 16000
    wav = tfio.audio.resample(wav, sample_rate, target_sample_rate)

    return wav

def load_wav_for_map_train(filename, label):
    return load_wav_16k_mono(filename), label


In [29]:
# Define YAMNet model
yamnet_model_handle = "https://tfhub.dev/google/yamnet/1"
yamnet_model = hub.load(yamnet_model_handle)
yamnet_model.trainable = True

In [31]:
train_ds = tf.data.Dataset.from_tensor_slices((train_filenames, train_targets))
train_ds = train_ds.map(load_wav_for_map_train)

# Create TensorFlow Dataset for testing
test_filenames = filtered_pd[filtered_pd['split'] == 'test']['filename']
test_targets = filtered_pd[filtered_pd['split'] == 'test']['target']

test_ds = tf.data.Dataset.from_tensor_slices((test_filenames, test_targets))

# Load WAV data for mapping
def load_wav_for_map_test(filename, label):
    return load_wav_16k_mono(filename), label

test_ds = test_ds.map(load_wav_for_map_test)
test_ds.element_spec

# Extract embeddings
def extract_embedding(wav_data, label):
    scores, embeddings, spectrogram = yamnet_model(wav_data)
    num_embeddings = tf.shape(embeddings)[0]
    return (embeddings,
            tf.repeat(label, num_embeddings))


train_ds = train_ds.map(extract_embedding).unbatch()
train_ds.element_spec

# Apply embedding extraction to testing dataset
test_ds = test_ds.map(extract_embedding).unbatch()
test_ds.element_spec

# Cache the datasets
cached_train_ds = train_ds.cache()
cached_test_ds = test_ds.cache()

# Shuffle, batch, and prefetch the datasets
train_ds = cached_train_ds.shuffle(1000).batch(64).prefetch(tf.data.AUTOTUNE)
test_ds = cached_test_ds.batch(32).prefetch(tf.data.AUTOTUNE)


In [23]:
my_model = Sequential([
    Input(shape=(1024,), dtype=tf.float32, name='input_embedding'),
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(len(my_classes), activation='softmax')
], name='my_model')

my_model.summary()

# Adjust the learning rate and compile the model
initial_learning_rate = 0.0001
lr_schedule = ExponentialDecay(
    initial_learning_rate, decay_steps=10000, decay_rate=0.9, staircase=True)

my_model.compile(loss='sparse_categorical_crossentropy',
                 optimizer=Adam(learning_rate=lr_schedule),
                 metrics=['accuracy'])


batch_size = 32
total_samples = len(train_filenames)
steps_per_epoch = total_samples // batch_size

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 1024)              1049600   
                                                                 
 batch_normalization_7 (Batc  (None, 1024)             4096      
 hNormalization)                                                 
                                                                 
 dropout_7 (Dropout)         (None, 1024)              0         
                                                                 
 dense_11 (Dense)            (None, 512)               524800    
                                                                 
 batch_normalization_8 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dropout_8 (Dropout)         (None, 512)               0  

In [25]:
history = my_model.fit(train_ds,
                       epochs=50,
                       steps_per_epoch=steps_per_epoch,
                       )

# Evaluate the model on the test set
loss, accuracy = my_model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

Epoch 1/50
7/7 [==============================] - 6s 226ms/step - loss: 1.1693 - accuracy: 0.6116
Epoch 2/50
7/7 [==============================] - 2s 237ms/step - loss: 1.2244 - accuracy: 0.5848
Epoch 3/50
7/7 [==============================] - 2s 266ms/step - loss: 1.3395 - accuracy: 0.5513
Epoch 4/50
7/7 [==============================] - 2s 128ms/step - loss: 1.4299 - accuracy: 0.5379
Epoch 5/50
7/7 [==============================] - 2s 269ms/step - loss: 1.1784 - accuracy: 0.6094
Epoch 6/50
7/7 [==============================] - 2s 334ms/step - loss: 1.0366 - accuracy: 0.6518
Epoch 7/50
7/7 [==============================] - 1s 213ms/step - loss: 1.3074 - accuracy: 0.5915
Epoch 8/50
7/7 [==============================] - 2s 275ms/step - loss: 1.4045 - accuracy: 0.5268
Epoch 9/50
7/7 [==============================] - 2s 267ms/step - loss: 1.3392 - accuracy: 0.5580
Epoch 10/50
7/7 [==============================] - 2s 256ms/step - loss: 1.0040 - accuracy: 0.6451
Epoch 11/50
7/7 [==

In [34]:
history = my_model.fit(train_ds,
                       epochs=100,
                       steps_per_epoch=steps_per_epoch,
                       )

# Evaluate the model on the test set
loss, accuracy = my_model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

Epoch 1/100
7/7 [==============================] - 9s 393ms/step - loss: 1.0730 - accuracy: 0.6116
Epoch 2/100
7/7 [==============================] - 3s 378ms/step - loss: 1.2891 - accuracy: 0.5893
Epoch 3/100
7/7 [==============================] - 3s 346ms/step - loss: 1.2577 - accuracy: 0.5915
Epoch 4/100
7/7 [==============================] - 3s 161ms/step - loss: 1.3819 - accuracy: 0.5379
Epoch 5/100
7/7 [==============================] - 2s 343ms/step - loss: 1.0542 - accuracy: 0.6138
Epoch 6/100
7/7 [==============================] - 3s 460ms/step - loss: 1.0530 - accuracy: 0.6384
Epoch 7/100
7/7 [==============================] - 2s 287ms/step - loss: 1.2872 - accuracy: 0.5848
Epoch 8/100
7/7 [==============================] - 3s 359ms/step - loss: 1.1318 - accuracy: 0.5915
Epoch 9/100
7/7 [==============================] - 3s 379ms/step - loss: 1.1466 - accuracy: 0.6228
Epoch 10/100
7/7 [==============================] - 3s 377ms/step - loss: 0.9575 - accuracy: 0.6652
Epoch 11/

181/181 [==============================] - 27s 142ms/step - loss: 0.9845 - accuracy: 0.6906
Loss:  0.9845073819160461
Accuracy:  0.6905588507652283


In [10]:
my_model.save("v_bird_model.h5")